In [7]:
%matplotlib inline
from pylab import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import networkx as nx
from networkx.drawing.nx_agraph import write_dot, graphviz_layout

In [8]:
import watertap as wt

In [9]:
# unit_process_name_list needs to be a list
unit_process_name_list = ['chlorination_twb']
T = wt.treatment_train_design.create_train(unit_process_name_list=unit_process_name_list,
                                            source_water_reference = 'Poseidon',
                                            source_water_type = 'Wastewater',
                                            source_water_case_study ='Typical untreated domestic wastewater',
                                            enduse_water_reference = 'Poseidon',
                                            enduse_water_type = 'Environmental',
                                            enduse_water_case_study = 'EPA: Environmental Reuse',
                                            flow = 0)

adding unit process to network: chlorination_twb


In [10]:
wt.display.show_train2(T)

In [11]:
T.nodes['source']['Flow'] = 12575*24 #37854.118 # sets flow. units in m3/d. 37854.118 m3/d = 10 million gallons per day 

In [13]:
M = wt.build_model(T)

assuming initial chlorine demand is 0


In [14]:
wt.run_model(model=M)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 16, 'Number of variables': 16, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.12\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.20965790748596191}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [15]:
analysis_variables = ['Flow', 'TOC' , 'Nitrate', 'Cost', 'TDS']
model_results = wt.post_processing.get_result_table(M, analysis_variables)
model_results

,Flow,TOC,Nitrate,Cost,TDS
chlorination_twb_feedwater,301800.0,140.0,0.0,0.000,0.0
chlorination_twb,301800.0,140.0,0.0,4.242,0.0
chlorination_twb_waste_stream,0.0,0.0,0.0,0.000,0.0
chlorination_twb_use_stream,301800.0,140.0,0.0,0.000,0.0


In [16]:
initial_chlorine_demand = 0
contact_time = 1.5  # hours
contact_time_mins = 1.5 * 60
ct = 450  # mg/L-min ---> ASSUME CALI STANDARD FOR NOW
chlorine_decay_rate = 3.0  # mg/Lh

In [17]:
applied_cl2_dose = (
    initial_chlorine_demand
    + chlorine_decay_rate * contact_time
    + ct / contact_time_mins
) #mg/L

In [18]:
applied_cl2_dose 

9.5

In [24]:
flow_test = 6248#*24

In [25]:
flow_test * 1000 * 9.5 / 1000000

59.356

In [7]:
#### LOAD TRAIN ####
path = 'Tutorial1_treatment_train_example.csv'
T = wt.load_train(path)

In [8]:
# display command
wt.display.show_train2(T)

##### Step 4: Add another unit process in parallel or series

In [9]:
T.nodes['source']['Flow'] = 100
M = wt.build_model(T)

In [10]:
wt.run_model(model=M)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 45, 'Number of variables': 51, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.12\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.0966958999633789}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [11]:
analysis_variables = ['Flow', 'TOC' , 'Nitrate', 'Cost', 'TDS']
model_results = wt.post_processing.get_result_table(M, analysis_variables)
model_results

,Flow,TOC,Nitrate,Cost,TDS
microfiltration_twb_feedwater,50.0,93.333,0.0,0.000,0.0
ultrafiltration_twb_feedwater,50.0,46.667,0.0,0.000,0.0
microfiltration_twb,50.0,93.333,0.0,0.033,0.0
ultrafiltration_twb,50.0,186.667,0.0,0.033,0.0
microfiltration_twb_waste_stream,2.5,0.000,0.0,0.000,0.0
microfiltration_twb_use_stream,47.5,93.333,0.0,0.000,0.0
chlorination_twb_feedwater,95.0,280.000,0.0,0.000,0.0
chlorination_twb,95.0,280.000,0.0,3.121,0.0
ultrafiltration_twb_waste_stream,2.5,0.000,0.0,0.000,0.0
ultrafiltration_twb_use_stream,47.5,186.667,0.0,0.000,0.0


In [7]:
##### ADD UNIT PROCESS IN PARALLEL ####
T = wt.treatment_train_design.add_unit_process(T, from_node = 'source',
                                                to_node = 'use', 
                                                unit_process_name_list = ['ultrafiltration_twb'],
                                                unit_process_list = ['ultrafiltration_twb'],
                                                order = 'parallel')

adding unit process to network: ultrafiltration_twb


In [8]:
wt.display.show_train2(T)

In [9]:
##### ADD UNIT PROCESS IN SERIES ####
T = wt.treatment_train_design.add_unit_process(T, from_node = 'use',
                                                to_node = None, 
                                                unit_process_name_list = ['chlorination_twb'],
                                                unit_process_list = ['chlorination_twb'],
                                                order = 'series')

adding unit process to network: chlorination_twb


In [10]:
wt.display.show_train(T)

##### 5. Add a recycle stream

In [10]:
T = wt.treatment_train_design.add_recycle_stream(T,
                       unit_process = 'microfiltration_twb',
                       unit_process_name = 'microfiltration_twb',
                       recyle_fraction_of_waste = 1,
                       number_of_affected_streams = 1)

NameError: name 'update_unit_process_recycle' is not defined

In [11]:
wt.display.show_train(T)

##### Step 6: Add multiple water sources

In [11]:
# ADD ANOTHER SOURCE. NEED TO PROVIDE A NEW SOURCE NAME.
source_to_node_names = [['source1', 'ultrafiltration_twb_start']]
T = wt.treatment_train_design.add_multiple_water_sources(T,
                                              number_of_additional_sources = 1,
                                              source_water_reference = 'Poseidon',
                                              source_water_type = 'Wastewater',
                                              source_water_case_study = 'Typical untreated domestic wastewater',
                                              source_to_node_names=source_to_node_names,
                                              flow = 0)

adding another water source: source1


In [12]:
wt.display.show_train(T)

##### Step 7: Add multiple end use water source

In [13]:
# ADD ANOTHER END USE 
node_to_use_names = [['chlorination_twb_end', 'use2']]
T = wt.treatment_train_design.add_multiple_water_enduses(T,
                                              number_of_additional_ends = 1,
                                              enduse_water_reference = 'Poseidon',
                                              enduse_water_type = 'Wastewater',
                                              enduse_water_case_study = 'Typical untreated domestic wastewater',
                                              node_to_end_names= node_to_use_names,
                                              min_flow = 0)

adding another water use: use2


In [14]:
wt.display.show_train(T)

##### Step 8: Add a stream from any point to any point. For this, user must define the type of stream being added. Options are: feedwater_stream, waste_stream, use_stream, recycle_stream, source_and_recycle_stream, treatment_process

In [15]:
for edge in T.edges():
    print(T.edges[edge]['name'], '----->>>' , T.edges[edge]['type'])

microfiltration_twb_feedwater ----->>> feedwater_stream
ultrafiltration_twb_feedwater ----->>> feedwater_stream
microfiltration_twb ----->>> treatment_process
microfiltration_twb_waste_stream ----->>> waste_stream
microfiltration_twb_use_stream ----->>> use_stream
chlorination_twb_feedwater ----->>> feedwater_stream
ultrafiltration_twb ----->>> treatment_process
ultrafiltration_twb_waste_stream ----->>> waste_stream
ultrafiltration_twb_use_stream ----->>> use_stream
chlorination_twb ----->>> treatment_process
chlorination_twb_waste_stream ----->>> waste_stream
chlorination_twb_use_stream ----->>> use_stream
chlorination_twb_end_treated_use2_stream ----->>> use_stream
source1_ultrafiltration_twb_start_feedwater ----->>> feedwater_stream


In [23]:
T = wt.treatment_train_design.add_stream(T, 
                                         from_node = 'microfiltration_twb_end', 
                                        to_node = 'use2',
                                        stream_name = 'treated_stream_example',
                                        stream_type = 'treated_stream') # TODO

In [24]:
wt.display.show_train(T)

##### Step 9: Remove unit process

In [25]:
T = wt.filter_processes.remove_single_process(T, process_name = 'ultrafiltration_twb')

In [26]:
wt.display.show_train(T)

##### Step 10: Save and load treatment train

In [16]:
#### SAVE TRAIN ####
path = 'Tutorial1_treatment_train_example.csv'
wt.save_train(T, path)

In [17]:
#### LOAD TRAIN ####
path = 'Tutorial1_treatment_train_example.csv'
TT = wt.load_train(path)

In [72]:
wt.display.show_train(T)

In [8]:
T.nodes['source']['Flow'] = 100
M = wt.build_model(T)
wt.run_model(model=M)

analysis_variables = ['Flow', 'TOC' , 'Nitrate', 'Cost', 'TDS']
model_results = wt.post_processing.get_result_table(M, analysis_variables)
model_results

In [ ]:
from src import data_processing
data_processing.cost_curve_data2equation(filename="../WaterTAP3CostCurves.xlsx", sheet_name="CostData")